In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
#################################
###### Variables Generales ######
num_lags = 15

In [4]:
arch_entrada = "datasets/emp3_sellout_transf.csv"
arch_salida = "datasets/emp3_sellout_train.csv"

In [5]:
def crear_nuevos_features(df_param):
    # Diferencia entre lo solicitado y lo entregado
    df_param["dif_cust_request_tn"]=df_param.cust_request_tn-df_param.tn
    df_param["dif_cust_request_tn_porc"]=np.where(df_param.cust_request_tn==0,0,100*df_param.dif_cust_request_tn/df_param.cust_request_tn)
   
    return df_param

In [6]:
def crear_lags_prod_cust(df_param):
    # Lags para product_id-customer_id
    for i in range(1, num_lags + 1):
        df_param[f'tn_lag_{i}'] = df_param.groupby(['product_id','customer_id'])['tn'].shift(i)

    return df_param

In [7]:
def crear_delta_lags_prod_cust(df_param):
    for i in range(1, num_lags+1):
        df_param[f'tn_delta_{i}'] = df_param.groupby(['product_id','customer_id'])['tn'].diff(i)

    return df_param

In [8]:
df = pd.read_csv(arch_entrada)

#Antes que nada, ordeno por prod-cust,periodo
df = df.sort_values(by=["prod_cust","periodo"],ascending=True)

df = crear_nuevos_features(df)
df = crear_lags_prod_cust(df)
df = crear_delta_lags_prod_cust(df)

df.to_csv(arch_salida, index=False)

## Verificaciones

In [9]:
# Nuevos features
df_control = df[df.dif_cust_request_tn_porc > 0]
df_control[["product_id","customer_id","periodo","tn","cust_request_tn","dif_cust_request_tn","dif_cust_request_tn_porc"]].sample(10)

,product_id,customer_id,periodo,tn,cust_request_tn,dif_cust_request_tn,dif_cust_request_tn_porc
10443,20074,10010,201807,-0.812260,1.808366,2.620625,144.916769
2910,20020,10171,201709,2.087634,2.171741,0.084107,3.872795
1708,20011,10084,201806,0.224431,0.997673,0.773243,77.504587
72674,20597,10010,201803,1.987000,2.551351,0.564351,22.119687
3018,20021,10002,201707,2.890301,3.133295,0.242995,7.755243
3038,20021,10002,201903,1.696571,1.782711,0.086140,4.831943
81931,20684,10001,201706,0.044699,0.239902,0.195203,81.367624
67673,20544,10001,201907,-1.238842,-1.480348,-0.241506,16.314110
26124,20201,10012,201808,1.557356,1.701799,0.144443,8.487641
52416,20407,10003,201703,0.162521,0.395863,0.233342,58.945192


In [10]:
# Lags prod-cust
df[["product_id","customer_id","periodo","tn","tn_lag_1","tn_lag_2","tn_lag_3"]].head(50)

,product_id,customer_id,periodo,tn,tn_lag_1,tn_lag_2,tn_lag_3
0,20001,10009,201701,-0.275718,NaN,NaN,NaN
1,20001,10009,201702,0.123594,-0.275718,NaN,NaN
2,20001,10009,201703,-0.330526,0.123594,-0.275718,NaN
3,20001,10009,201704,-0.894260,-0.330526,0.123594,-0.275718
4,20001,10009,201705,0.867411,-0.894260,-0.330526,0.123594
5,20001,10009,201706,1.657493,0.867411,-0.894260,-0.330526
6,20001,10009,201707,-1.090002,1.657493,0.867411,-0.894260
7,20001,10009,201708,2.840482,-1.090002,1.657493,0.867411
8,20001,10009,201709,-1.090002,2.840482,-1.090002,1.657493
9,20001,10009,201710,-1.073630,-1.090002,2.840482,-1.090002


In [11]:
# Delta Lags prod-cust
df[["product_id","customer_id","periodo","tn","tn_lag_1","tn_delta_1","tn_lag_2","tn_delta_2","tn_lag_3","tn_delta_3"]].head(50)

,product_id,customer_id,periodo,tn,tn_lag_1,tn_delta_1,tn_lag_2,tn_delta_2,tn_lag_3,tn_delta_3
0,20001,10009,201701,-0.275718,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,10009,201702,0.123594,-0.275718,0.399313,NaN,NaN,NaN,NaN
2,20001,10009,201703,-0.330526,0.123594,-0.454120,-0.275718,-0.054807,NaN,NaN
3,20001,10009,201704,-0.894260,-0.330526,-0.563734,0.123594,-1.017854,-0.275718,-0.618542
4,20001,10009,201705,0.867411,-0.894260,1.761671,-0.330526,1.197936,0.123594,0.743816
5,20001,10009,201706,1.657493,0.867411,0.790083,-0.894260,2.551753,-0.330526,1.988019
6,20001,10009,201707,-1.090002,1.657493,-2.747495,0.867411,-1.957412,-0.894260,-0.195742
7,20001,10009,201708,2.840482,-1.090002,3.930484,1.657493,1.182989,0.867411,1.973072
8,20001,10009,201709,-1.090002,2.840482,-3.930484,-1.090002,0.000000,1.657493,-2.747495
9,20001,10009,201710,-1.073630,-1.090002,0.016371,2.840482,-3.914113,-1.090002,0.016371


######### Agregar cuando se pongan lags de prod, cust y cats

for i in range(1, num_lags + 1):
    df_cat1[f'cat1_tn_lag_{i}'] = df_cat1.groupby('cat1')['cat1_tn'].shift(i)
    df_cat1[f'cat1_tn_delta_{i}'] = df_cat1.groupby('cat1')['cat1_tn'].diff(i)

df_cat1.head()

df = df.merge(df_cat1,how="inner",on=["cat1","periodo"])
df.head()

df[["product_id","cat1","periodo","cat1_tn","cat1_tn_lag_1","cat1_tn_delta_1","cat1_tn_lag_2","cat1_tn_delta_2","cat1_tn_lag_3","cat1_tn_delta_3"]].sort_values(by=["product_id","periodo"],ascending=True).head(10)

df_cat1_hc = df_cat1[df_cat1.cat1=="HC"]


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133681 entries, 0 to 133680
Data columns (total 60 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   product_id                 133681 non-null  int64  
 1   periodo                    133681 non-null  int64  
 2   customer_id                133681 non-null  int64  
 3   cero_ventas                133681 non-null  int64  
 4   plan_precios_cuidados      133681 non-null  float64
 5   cust_request_qty           133681 non-null  float64
 6   cust_request_tn            133681 non-null  float64
 7   tn                         133681 non-null  float64
 8   periodo_fecha              133681 non-null  object 
 9   mes                        133681 non-null  int64  
 10  meses_historia_prod        133681 non-null  int64  
 11  meses_historia_cliente     133681 non-null  int64  
 12  cliente_estrella           133681 non-null  int64  
 13  prod_cust                  13

In [13]:
df.isna().sum()

product_id                       0
periodo                          0
customer_id                      0
cero_ventas                      0
plan_precios_cuidados            0
cust_request_qty                 0
cust_request_tn                  0
tn                               0
periodo_fecha                    0
mes                              0
meses_historia_prod              0
meses_historia_cliente           0
cliente_estrella                 0
prod_cust                        0
cat1                             0
cat2                             0
cat3                             0
brand                            0
sku_size                         0
producto_estrella                0
temp_media                       0
temp_max_media                   0
temp_min_media                   0
IPC                              0
promedio_mens_dolar_venta        0
catastrofe                       0
tn_mas_2                      9971
tn_mas_2_original             9971
dif_cust_request_tn 

In [14]:
len(df.columns)

60